In [238]:
import glob
import os
import pandas as pd

In [239]:
caminho_raw = "../dados/raw/"

In [240]:
arquivos_bacen_resumo = "../dados/bacen/resumo_bacen/resumo_*.csv"
arquivos_bacen_dre = "../dados/bacen/dre_bacen/dre_*.csv"

lista_arquivo_resumo = glob.glob(arquivos_bacen_resumo)
lista_arquivo_dre = glob.glob(arquivos_bacen_dre)

In [241]:
lista_resumo = []
lista_dre = []

for arquivo_resumo in lista_arquivo_resumo:
    nome_limpo_resumo = os.path.basename(arquivo_resumo)
    data_periodo_resumo = nome_limpo_resumo.split('_')[1].replace('.csv','')

    temp_df_resumo = pd.read_csv(arquivo_resumo, sep=";", encoding='utf-8', low_memory=False)

    lista_resumo.append(temp_df_resumo)

df_resumo_total = pd.concat(lista_resumo, ignore_index=True)

for arquivo_dre in lista_arquivo_dre:
    nome_limpo_dre = os.path.basename(arquivo_dre)
    data_periodo_dre = nome_limpo_dre.split('_')[1].replace('.csv','')

    temp_df_dre = pd.read_csv(arquivo_dre, sep=";", encoding='utf-8', low_memory=False)

    lista_dre.append(temp_df_dre)

df_dre_total = pd.concat(lista_dre, ignore_index=True)

df_resumo_total.columns = df_resumo_total.columns.str.strip()
df_dre_total.columns = df_dre_total.columns.str.strip()

df_resumo_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18017 entries, 0 to 18016
Data columns (total 24 columns):
 #   Column                                                                          Non-Null Count  Dtype  
---  ------                                                                          --------------  -----  
 0   Instituição                                                                     18017 non-null  object 
 1   Código                                                                          17170 non-null  float64
 2   Conglomerado Prudencial                                                         8373 non-null   object 
 3   Conglomerado Financeiro                                                         2600 non-null   float64
 4   Conglomerado Prudencial.1                                                       8374 non-null   float64
 5   TCB                                                                             17170 non-null  object 
 6   TC            

In [242]:
df_resumo = df_resumo_total.copy()

df_resumo['Carteira de Crédito'] = df_resumo['Carteira de Crédito'].fillna(df_resumo['Carteira de Crédito Classificada'])
df_resumo['Passivo Exigível'] = df_resumo['Passivo Exigível'].fillna(df_resumo['Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros'])

colunas_excluir = ['Carteira de Crédito Classificada','Conglomerado Prudencial.1', 'Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros', 'Títulos e Valores Mobiliários', 'Unnamed: 19', 'Unnamed: 20', 'Conglomerado Financeiro']

df_resumo = df_resumo.drop(columns=colunas_excluir)

df_resumo = df_resumo.dropna(subset='Conglomerado Prudencial')

df_resumo.to_parquet(os.path.join(caminho_raw, "resumo.parquet"), index=False)